In [42]:
from pyspark.sql import SparkSession, functions as f, types as t

In [43]:
spark = SparkSession.builder.master('local[3]').getOrCreate()

In [44]:
print(spark.version)

3.3.0


In [45]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [46]:
film_category_df.select(
    'film_id',
    'category_id'
).groupBy(
    'category_id'
).agg(
    f.countDistinct('film_id').alias('films_qtt')
).join(
    category_df, 'category_id', 'inner'
).select(
    'name',
    'films_qtt'
).orderBy(
    f.desc('films_qtt')
).show()

+-----------+---------+
|       name|films_qtt|
+-----------+---------+
|     Sports|       74|
|    Foreign|       73|
|     Family|       69|
|Documentary|       68|
|  Animation|       66|
|     Action|       64|
|        New|       63|
|      Drama|       62|
|      Games|       61|
|     Sci-Fi|       61|
|   Children|       60|
|     Comedy|       58|
|     Travel|       57|
|   Classics|       57|
|     Horror|       56|
|      Music|       51|
+-----------+---------+



  2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [47]:
rental_df.join(
    inventory_df, 'inventory_id', 'inner'
).join(
    film_actor_df, 'film_id', 'inner'
).join(
    actor_df, 'actor_id', 'inner'
).select(
    'actor_id',
    'rental_id',
    f.concat_ws(' ','first_name', 'last_name').alias('full_name'),
).groupBy(
    'actor_id',
    'full_name'
).agg(
    f.countDistinct('rental_id').alias('times_rented')
).orderBy(
    f.desc('times_rented')
).show(10)

+--------+------------------+------------+
|actor_id|         full_name|times_rented|
+--------+------------------+------------+
|     107|    GINA DEGENERES|         753|
|     181|    MATTHEW CARREY|         678|
|     198|       MARY KEITEL|         674|
|     144|ANGELA WITHERSPOON|         654|
|     102|       WALTER TORN|         640|
|      60|       HENRY BERRY|         612|
|     150|       JAYNE NOLTE|         611|
|      37|        VAL BOLGER|         605|
|      23|     SANDRA KILMER|         604|
|      90|      SEAN GUINESS|         599|
+--------+------------------+------------+
only showing top 10 rows



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [48]:
rental_df.alias(
    'rent'
).join(
    payment_df.alias('pay'), 'rental_id', 'inner'
).join(
    inventory_df.alias('inv'), 'inventory_id', 'inner'
).join(
    film_category_df.alias('film_cat'), 'film_id', 'inner'
).join(
    category_df.alias('cat'), 'category_id', 'inner'
).select(
    'pay.rental_id',
    'inv.film_id',
    'cat.name',
    'amount'
).groupBy(
    'name'
).agg(
    f.round(f.sum('amount'),2).alias('total_spent')
).orderBy(
    f.desc('total_spent')
).show(1)

+------+-----------+
|  name|total_spent|
+------+-----------+
|Sports|    5314.21|
+------+-----------+
only showing top 1 row



4.
Вивести назви фільмів, яких не має в inventory.

In [49]:
film_df.join(
    inventory_df.alias('i'), 'film_id', 'left'
).where(
    'i.film_id is null'
).select(
    'title'
).distinct(
).orderBy(
    'title'
).show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [50]:
film_df.alias('film').join(
    film_category_df.alias('film_cat'), 'film_id', 'inner'
).join(
    category_df.alias('cat'), 'category_id', 'inner'
).join(
    film_actor_df.alias('film_act'), 'film_id', 'inner'
).join(
    actor_df.alias('act'), 'actor_id', 'inner'
).where(
    'cat.name  = "Children"'
).select(
    'film.film_id',
    f.concat_ws(' ', 'first_name', 'last_name').alias('full_name')
).groupBy(
    'full_name'
).agg(
    f.countDistinct('film_id').alias('films_qtt')
).orderBy(
    f.desc('films_qtt')
).show(3)

+------------+---------+
|   full_name|films_qtt|
+------------+---------+
|HELEN VOIGHT|        7|
| SUSAN DAVIS|        6|
|  MARY TANDY|        5|
+------------+---------+
only showing top 3 rows



Stop Spark session:

In [51]:
spark.stop()